In [312]:
input = '''
805A
964A
459A
968A
671A
'''.strip()

In [313]:
# First, some definitions

codes = input.split('\n')

directional_keypad = [['', '^', 'A'],
                      ['<', 'v', '>']]
numeric_keypad = [['7', '8', '9'],
                  ['4', '5', '6'],
                  ['1', '2', '3'],
                  ['', '0', 'A']]

In [314]:
symbols = {(0, -1): '^', (0, 1): 'v', (-1, 0): '<', (1, 0): '>'}
def bfs(start, end, keypad):
    best_paths = []
    best_path_len = float('inf')

    queue = [(start, '')]
    
    while queue:
        (x, y), path = queue.pop(0)

        if (x, y) == end:
            if len(path) < best_path_len:
                best_paths = [path + 'A']
                best_path_len = len(path)
            elif len(path) == best_path_len:
                best_paths.append(path + 'A')
            continue

        if len(path) >= best_path_len:
            continue
        
        for dx, dy in [(0, 1), (0, -1), (-1, 0), (1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(keypad) and 0 <= ny < len(keypad[0]) and keypad[nx][ny] != '':
                queue.append(((nx, ny), path + symbols[dy, dx]))

    return best_paths

def get_all_paths(keypad):
    paths = {}
    for x1 in range(len(keypad)):
        for y1 in range(len(keypad[0])):
            if keypad[x1][y1] != '':
                for x2 in range(len(keypad)):
                    for y2 in range(len(keypad[0])):
                        if keypad[x2][y2] != '':
                            paths[keypad[x1][y1], keypad[x2][y2]] = bfs((x1, y1), (x2, y2), keypad)
    return paths

# We pre-calculate all the minimum paths between all pairs of keys for both keypads
numeric_keypad_paths = get_all_paths(numeric_keypad)
directional_keypad_paths = get_all_paths(directional_keypad)
all_shortest_paths = numeric_keypad_paths | directional_keypad_paths

In [315]:
# Now we use the previously calculated paths to calculate the minimum path length for the given code. We do this recursively, with memoization.
# Base Case: The human punches the code (bot = 0). The length of the path is the length of the code.
# Recurrence: For each pair of symbols in the code, for each possible (pre-calculated) path, we calculate the minimum path length for the next bot.
# Result: We keep the minimum path length once all bots have punched the code.

from functools import cache

@cache
def get_length_for_shortest_path(code, bot):

    # The human just punches in the code
    if bot == 0:
        return len(code)
    
    # The iterations are the pairs of symbols in the code
    iterations = zip(f"A{code}", code)

    # Sum the total for the minimum path length for each pair of symbols
    total = 0
    for (start, end) in iterations:

        min_path_len = float('inf')
        for path in all_shortest_paths[start, end]:
            length = get_length_for_shortest_path(path, bot - 1)
            if length < min_path_len:
                min_path_len = length

        total += min_path_len
    return total

In [316]:
# Part 1
total = 0
for code in codes:
    total += get_length_for_shortest_path(code, 3) * int(code[:-1])
total

278748

In [317]:
# Part 2
total = 0
for code in codes:
    total += get_length_for_shortest_path(code, 26) * int(code[:-1])
total

337744744231414